In [1]:
import torch
import torch.nn as nn
import numpy as np
import os
import sys

In [2]:
activations = torch.load('variables/latents_celeba_vgg16.pt', weights_only=True)
labels = torch.load('variables/labels.pt', weights_only=True)
activations.shape, labels.shape

(torch.Size([20260, 512]), torch.Size([20260, 40]))

In [16]:
from models.linear_cav import LinearCAV

model1 = LinearCAV(40, 512, device='cpu')
list(model1.state_dict())

['weights', 'biases']

In [17]:
list(model1.parameters())

[Parameter containing:
 tensor([[-1.6655,  0.1851,  0.7804,  ..., -1.1509,  1.9469, -0.3770],
         [ 0.9161,  2.3566, -1.1967,  ...,  1.1529,  0.2808,  1.1489],
         [-1.0718,  0.0517, -1.3662,  ..., -0.2648, -1.6436, -0.0631],
         ...,
         [-0.3991, -1.0432,  0.2954,  ...,  0.8221,  1.7303, -0.3322],
         [-1.3112, -0.1184,  0.0181,  ...,  1.0871, -0.7002, -0.4613],
         [ 0.5916, -0.9506,  0.5127,  ..., -0.4886, -1.5401, -0.9758]],
        requires_grad=True),
 Parameter containing:
 tensor([[ 1.0766e+00, -2.0249e-01, -1.2705e+00, -1.2950e-01,  2.3764e-02,
          -2.2701e-01, -1.2777e+00,  5.9295e-01,  2.3917e-01, -4.0568e-01,
           3.0885e-01,  6.7448e-01, -1.0682e+00, -9.9402e-01,  3.6662e-01,
           5.4662e-02,  2.4230e+00, -2.0279e-01,  1.0643e+00, -1.2756e-02,
          -6.2723e-01, -2.1509e+00, -1.4944e+00,  3.7717e-01, -5.0100e-01,
          -1.5439e+00, -2.1828e+00,  3.0414e-01,  2.9599e-01,  1.3980e+00,
           8.7687e-01, -1.6100e+00

In [18]:
cavs1 = torch.load('results/cavs-linear:alpha0_lr0.1/cavs.pt', weights_only=True)
model1.weights.data = cavs1

In [19]:
torch.allclose(cavs1, model1.weights.data)

True

In [20]:
torch.save(model1.state_dict(), 'checkpoints/lcav_vgg16_celeba.pth')

In [4]:
from utils.cav import compute_all_cavs

scavs = compute_all_cavs(activations, labels)
scavs.shape

torch.Size([40, 512])

In [ ]:
torch.linalg.norm(scavs, dim=1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000])

In [7]:
from models.signal_cav import SignalCAV

model = SignalCAV(scavs, torch.randn(scavs.shape), device='cpu')

In [8]:
torch.save(model.state_dict(), 'checkpoints/scav_vgg16_celeba.pth')

In [15]:
dict = torch.load("checkpoints/scav_vgg16_celeba.pth", weights_only=True)
dict['weights'].shape

torch.Size([1, 40, 512])

In [2]:
latents1 = torch.load("variables/latents_diffae.pt", weights_only=True)
latents2 = torch.load("variables/latents.pt", weights_only=True)
latents3 = torch.load("variables/latents_celeba_vgg16.pt", weights_only=True)

In [3]:
print(latents1.shape)
print(latents2.shape)
print(latents3.shape)

torch.Size([202599, 512])
torch.Size([20260, 512])
torch.Size([20260, 512])


In [5]:
labels1 = torch.load("variables/labels_diffae.pt", weights_only=True)
labels2 = torch.load("variables/labels.pt", weights_only=True)
labels1[:5], labels2[:5]

(tensor([[-1.,  1., -1., -1., -1., -1., -1., -1., -1.,  1., -1., -1., -1., -1.,
          -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.,  1., -1., -1., -1.,
          -1., -1., -1., -1., -1., -1., -1., -1.,  1., -1., -1.,  1.],
         [-1., -1., -1., -1., -1., -1.,  1.,  1.,  1., -1., -1., -1., -1., -1.,
          -1., -1., -1., -1., -1.,  1.,  1.,  1., -1., -1.,  1.,  1., -1., -1.,
           1., -1., -1.,  1., -1., -1.,  1., -1., -1., -1., -1.,  1.],
         [-1.,  1.,  1.,  1., -1.,  1.,  1.,  1., -1., -1., -1.,  1., -1., -1.,
          -1., -1., -1., -1., -1.,  1.,  1.,  1., -1., -1.,  1., -1., -1., -1.,
          -1., -1., -1.,  1., -1., -1., -1., -1., -1., -1., -1.,  1.],
         [-1.,  1.,  1., -1., -1.,  1., -1., -1., -1., -1., -1., -1., -1., -1.,
          -1., -1., -1., -1.,  1., -1., -1., -1., -1., -1.,  1., -1.,  1., -1.,
          -1., -1., -1., -1.,  1., -1.,  1., -1.,  1., -1., -1.,  1.],
         [ 1., -1.,  1., -1., -1., -1., -1., -1.,  1., -1., -1., -1.,  1., -